In [1]:
# -*- coding: utf-8 -*-
"""
Osciloscopio Tektronix TDS1002B
Manual U (web): https://github.com/hgrecco/labosdf-bin/raw/master/manuals/TDS1002 Manual.pdf
Manual P (web): https://github.com/hgrecco/labosdf-bin/raw/master/manuals/TDS 100-1000-2000_prog.pdf
Manual U (local): \\Srvlabos\manuales\Tektronix\TDS1002 Manual.pdf
Manual P (local): \\Srvlabos\manuales\Tektronix\TDS 200-1000-2000_prog.pdf
"""

from __future__ import division, unicode_literals, print_function, absolute_import

import time

from matplotlib import pyplot as plt
import numpy as np
import visa

print(__doc__)






Osciloscopio Tektronix TDS1002B
Manual U (web): https://github.com/hgrecco/labosdf-bin/raw/master/manuals/TDS1002 Manual.pdf
Manual P (web): https://github.com/hgrecco/labosdf-bin/raw/master/manuals/TDS 100-1000-2000_prog.pdf
Manual U (local): \Srvlabos\manuales\Tektronix\TDS1002 Manual.pdf
Manual P (local): \Srvlabos\manuales\Tektronix\TDS 200-1000-2000_prog.pdf



In [5]:
help(rm.open_resource)

Help on method open_resource in module pyvisa.highlevel:

open_resource(self, resource_name, access_mode=<AccessModes.no_lock: 0>, open_timeout=0, resource_pyclass=None, **kwargs) method of pyvisa.highlevel.ResourceManager instance
    Return an instrument for the resource name.
    
    :param resource_name: name or alias of the resource to open.
    :param access_mode: access mode.
    :type access_mode: :class:`pyvisa.constants.AccessModes`
    :param open_timeout: time out to open.
    :param resource_pyclass: resource python class to use to instantiate the Resource.
                             Defaults to None: select based on the resource name.
    :param kwargs: keyword arguments to be used to change instrument attributes
                   after construction.
    
    :rtype: :class:`pyvisa.resources.Resource`



In [4]:
resource_name = 'USB0::0x0699::0x0363::C065092::INSTR'
rm = visa.ResourceManager()
osci = rm.open_resource(resource_name)

Exception: failed to set configuration
 [Errno 16] Resource busy

In [5]:
# Este string determina el intrumento que van a usar.
# Lo tienen que cambiar de acuerdo a lo que tengan conectado.
resource_name = 'USB0::0x0699::0x0363::C065092::INSTR'


rm = visa.ResourceManager()

osci = rm.open_resource(resource_name)

osci.query('*IDN?')

# Le pido algunos parametros de la pantalla, para poder escalear adecuadamente
xze, xin, yze, ymu, yoff = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')

# Modo de transmision: Binario
osci.write('DAT:ENC RPB')
osci.write('DAT:WID 1')

# Adquiere los datos del canal 1 y los devuelve en un array de numpy
data = osci.query_binary_values('CURV?', datatype='B', container=np.array)

tiempo = xze + np.arange(len(data)) * xin

plt.plot(tiempo, data)
plt.xlabel('Tiempo [s]')
plt.ylabel('Voltaje [V]')


ValueError: Please install PyUSB to use this resource type.
No module named usb

In [ ]:
# Si vas a repetir la adquisicion muchas veces sin cambiar la escala
# es util definir una funcion que mida y haga las cuentas
def definir_medir(inst):
    xze, xin, yze, ymu, yoff = inst.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')

    # creamos una function auxiliar
    def _medir():
        # Adquiere los datos del canal 1 y los devuelve en un array de numpy
        data = inst.query_binary_values('CURV?', datatype='B', container=np.array)

        tiempo = xze + np.arange(len(data)) * xin
        return tiempo, data
    
    # Devolvemos la funcion auxiliar que "sabe" la escala
    return _medir


medir = definir_medir(osci)
for n in range(10):
    tiempo, data = medir()
    plt.figure()
    plt.plot(tiempo, data)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Voltaje [V]')
    time.sleep(.1)

osci.close()

In [ ]:
#Getting data from Arduino
import serial

try:
    arduino = serial.Serial(*/dev/ttyACM*, timeout=1)
    
except:
    print 'Please check the port'

data = []
count = 0

